# Merge and Analyze Coinone USDT Monthly Candles

This notebook merges monthly CSVs into a single DataFrame, sorts chronologically, lets you exclude full KST dates from analysis, and computes median/average of the `close` column.


In [15]:
import os
import glob
import pandas as pd

pd.set_option("display.max_rows", 200)

# Adjust if needed
# DATA_DIR = "../data/coinone/monthly/usdt"
# PATTERN = "usdt_1m_*.csv.gz"  # expects gzip-compressed CSVs created by downloader
DATA_DIR = "../data/coinone/usdt/5m"
PATTERN = "usdt_5m_*.csv.gz"

all_files = sorted(glob.glob(os.path.join(DATA_DIR, PATTERN)))
print(f"Found {len(all_files)} files")
for f in all_files[:]:
    print("-", os.path.basename(f))

Found 22 files
- usdt_5m_2023-11.csv.gz
- usdt_5m_2023-12.csv.gz
- usdt_5m_2024-01.csv.gz
- usdt_5m_2024-02.csv.gz
- usdt_5m_2024-03.csv.gz
- usdt_5m_2024-04.csv.gz
- usdt_5m_2024-05.csv.gz
- usdt_5m_2024-06.csv.gz
- usdt_5m_2024-07.csv.gz
- usdt_5m_2024-08.csv.gz
- usdt_5m_2024-09.csv.gz
- usdt_5m_2024-10.csv.gz
- usdt_5m_2024-11.csv.gz
- usdt_5m_2024-12.csv.gz
- usdt_5m_2025-01.csv.gz
- usdt_5m_2025-02.csv.gz
- usdt_5m_2025-03.csv.gz
- usdt_5m_2025-04.csv.gz
- usdt_5m_2025-05.csv.gz
- usdt_5m_2025-06.csv.gz
- usdt_5m_2025-07.csv.gz
- usdt_5m_2025-08.csv.gz


# merge all csv 

In [16]:
# Merge files into a single DataFrame, ensuring parsed UTC timestamps

if not all_files:
    raise SystemExit("No files found. Check DATA_DIR and PATTERN.")

# Read with parsing to ensure 'timestamp_utc' is datetime (as produced by downloader)
# Fallback: if reading generic, still parse when column exists
frames = []
for path in all_files:
    try:
        df = pd.read_csv(path, compression="gzip", parse_dates=["timestamp_utc"], dtype=str)
    except (pd.errors.ParserError, ValueError, OSError, UnicodeError):
        df = pd.read_csv(path, compression="gzip")
        if "timestamp_utc" in df.columns:
            df["timestamp_utc"] = pd.to_datetime(df["timestamp_utc"], utc=True, errors="coerce")
    frames.append(df)

merged_df = pd.concat(frames, ignore_index=True)

# Sort by utc timestamp if available, otherwise try 'date' column
if "timestamp_utc" in merged_df.columns:
    merged_df = merged_df.drop_duplicates(subset=["timestamp_utc"]).sort_values("timestamp_utc").reset_index(drop=True)
elif "date" in merged_df.columns:
    merged_df["date"] = pd.to_datetime(merged_df["date"], errors="coerce")
    merged_df = merged_df.dropna(subset=["date"]).sort_values("date").reset_index(drop=True)
else:
    raise SystemExit("Neither 'timestamp_utc' nor 'date' column found.")

print(merged_df.shape)
merged_df.head()


(181868, 11)


,timestamp_kst,market,timeframe,open,high,low,close,volume,timestamp_utc,ts_ms,_month_kst
0,2023-11-29 16:50:00,USDT/KRW,5m,1289.0,1289.0,1289.0,1289.0,0.0,2023-11-29 07:50:00+00:00,1701244200000,2023-11
1,2023-11-29 16:55:00,USDT/KRW,5m,1289.0,1289.0,1289.0,1289.0,0.0,2023-11-29 07:55:00+00:00,1701244500000,2023-11
2,2023-11-29 17:00:00,USDT/KRW,5m,1289.0,1289.0,1289.0,1289.0,0.0,2023-11-29 08:00:00+00:00,1701244800000,2023-11
3,2023-11-29 17:05:00,USDT/KRW,5m,1289.0,1289.0,1289.0,1289.0,0.0,2023-11-29 08:05:00+00:00,1701245100000,2023-11
4,2023-11-29 17:10:00,USDT/KRW,5m,1289.0,1289.0,1289.0,1289.0,0.0,2023-11-29 08:10:00+00:00,1701245400000,2023-11


# Convert UTC to KST date to enable full-day exclusion in KST


In [17]:

if "timestamp_utc" in merged_df.columns:
    # Ensure 'timestamp_utc' is a timezone-aware datetime
    if not pd.api.types.is_datetime64tz_dtype(merged_df["timestamp_utc"]):
        merged_df["timestamp_utc"] = pd.to_datetime(merged_df["timestamp_utc"], utc=True, errors="coerce")
    # Now safe to use .dt.tz_convert
    kst = merged_df["timestamp_utc"].dt.tz_convert("Asia/Seoul")
    merged_df["date_kst"] = kst.dt.date
else:
    # If only generic 'date' exists, assume it's timezone-naive local time; adapt as needed
    merged_df["date_kst"] = pd.to_datetime(merged_df["date"], errors="coerce").dt.date

merged_df[["timestamp_utc", "date_kst", "close"]].head()


/var/folders/4q/54t22rpj7ngdvjyv_g50m2dr0000gn/T/ipykernel_8240/2688033363.py:3: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if not pd.api.types.is_datetime64tz_dtype(merged_df["timestamp_utc"]):


,timestamp_utc,date_kst,close
0,2023-11-29 07:50:00+00:00,2023-11-29,1289.0
1,2023-11-29 07:55:00+00:00,2023-11-29,1289.0
2,2023-11-29 08:00:00+00:00,2023-11-29,1289.0
3,2023-11-29 08:05:00+00:00,2023-11-29,1289.0
4,2023-11-29 08:10:00+00:00,2023-11-29,1289.0


# Exclude specific data

In [18]:
# Exclude by KST 1-hour windows and/or full KST dates
# - Windows: e.g., "2024-01-03_10:00" excludes [10:00, 11:00) on that day
# - Dates: e.g., "2024-02-13" excludes the entire day

# exclude_windows = ["2024-01-03_10:00"]

## note
# 2023-12-01_11:00, 2023-12-01_12:00 close 이상무
# 2023-12-14_12:00 close 이상무
# 2024-01-27_00:00 -> close는 아무 이상 없음

exclude_dates = ["2023-11-29", "2023-11-30"] # initial period
# exclude_windows = ["2024-04-14_05:00" "2024-03-06_04:00", "2024-04-22_15:00"]
exclude_windows = ["2024-04-14_05:00"]
# exclude_windows = []

In [19]:
# Parse KST datetime starts
raw_starts = pd.to_datetime(exclude_windows, format="%Y-%m-%d_%H:%M", errors="coerce")
raw_starts = raw_starts[~raw_starts.isna()]
if len(raw_starts) > 0:
    starts_kst = raw_starts.tz_localize("Asia/Seoul")
else:
    starts_kst = raw_starts

# Ensure required columns and types
if "timestamp_utc" not in merged_df.columns:
    raise SystemExit("Expected 'timestamp_utc' column for KST exclusion")

if not pd.api.types.is_datetime64tz_dtype(merged_df["timestamp_utc"]):
    merged_df["timestamp_utc"] = pd.to_datetime(merged_df["timestamp_utc"], utc=True, errors="coerce")
    merged_df = merged_df.dropna(subset=["timestamp_utc"]).reset_index(drop=True)

if "date_kst" not in merged_df.columns:
    merged_df["date_kst"] = merged_df["timestamp_utc"].dt.tz_convert("Asia/Seoul").dt.date

kst_ts = merged_df["timestamp_utc"].dt.tz_convert("Asia/Seoul")

# 1) Window-based exclusion mask
mask_window = pd.Series(False, index=merged_df.index)
for start in starts_kst:
    end = start + pd.Timedelta(hours=1)
    mask_window |= (kst_ts >= start) & (kst_ts < end)

# 2) Full-day exclusion mask based on KST date
parsed_dates = pd.to_datetime(exclude_dates, errors="coerce") if exclude_dates else pd.to_datetime([])
if hasattr(parsed_dates, "isna"):
    parsed_dates = parsed_dates[~parsed_dates.isna()]
exclude_date_objs = set(pd.Series(parsed_dates).dt.date.tolist()) if len(parsed_dates) > 0 else set()
mask_day = merged_df["date_kst"].isin(exclude_date_objs) if len(exclude_date_objs) > 0 else pd.Series(False, index=merged_df.index)

# Combine masks
mask_excluded = mask_window | mask_day
filtered_df = merged_df.loc[~mask_excluded].copy()

print({
    "total_rows": len(merged_df),
    "excluded_rows": int(mask_excluded.sum()),
    "excluded_window_rows": int(mask_window.sum()),
    "excluded_day_rows": int(mask_day.sum()),
    "remaining_rows": len(filtered_df),
    "excluded_windows": [f"{s.strftime('%Y-%m-%d %H:%M')}~{(s + pd.Timedelta(hours=1)).strftime('%H:%M')} KST" for s in starts_kst],
    "excluded_dates": sorted([d.isoformat() for d in exclude_date_objs]),
})
filtered_df.head()

{'total_rows': 181868, 'excluded_rows': 386, 'excluded_window_rows': 12, 'excluded_day_rows': 374, 'remaining_rows': 181482, 'excluded_windows': ['2024-04-14 05:00~06:00 KST'], 'excluded_dates': ['2023-11-29', '2023-11-30']}


/var/folders/4q/54t22rpj7ngdvjyv_g50m2dr0000gn/T/ipykernel_8240/902665243.py:13: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if not pd.api.types.is_datetime64tz_dtype(merged_df["timestamp_utc"]):


,timestamp_kst,market,timeframe,open,high,low,close,volume,timestamp_utc,ts_ms,_month_kst,date_kst
374,2023-12-01 00:00:00,USDT/KRW,5m,1337.0,1337.0,1337.0,1337.0,7.9984,2023-11-30 15:00:00+00:00,1701356400000,2023-12,2023-12-01
375,2023-12-01 00:05:00,USDT/KRW,5m,1335.0,1352.0,1330.0,1352.0,1091.69493872,2023-11-30 15:05:00+00:00,1701356700000,2023-12,2023-12-01
376,2023-12-01 00:10:00,USDT/KRW,5m,1348.0,1348.0,1348.0,1348.0,3.70919881,2023-11-30 15:10:00+00:00,1701357000000,2023-12,2023-12-01
377,2023-12-01 00:15:00,USDT/KRW,5m,1348.0,1350.0,1348.0,1350.0,996.2002563,2023-11-30 15:15:00+00:00,1701357300000,2023-12,2023-12-01
378,2023-12-01 00:20:00,USDT/KRW,5m,1350.0,1350.0,1350.0,1350.0,0.0,2023-11-30 15:20:00+00:00,1701357600000,2023-12,2023-12-01


# [debug] Filter by KST datetime range [start, end)

In [20]:

# Example: 2024-04-14_05:00 to 2024-04-14_06:00
range_start_kst_str = "2025-01-02_04:00"
range_end_kst_str = "2025-01-02_05:00"

if "timestamp_utc" not in filtered_df.columns:
    raise SystemExit("Expected 'timestamp_utc' in filtered_df")

# Parse inputs as KST-aware times
range_start_kst = pd.to_datetime(range_start_kst_str, format="%Y-%m-%d_%H:%M", errors="coerce").tz_localize("Asia/Seoul")
range_end_kst = pd.to_datetime(range_end_kst_str, format="%Y-%m-%d_%H:%M", errors="coerce").tz_localize("Asia/Seoul")

# Convert filtered_df timestamps to KST and filter
kst_ts = filtered_df["timestamp_utc"].dt.tz_convert("Asia/Seoul")
mask_range = (kst_ts >= range_start_kst) & (kst_ts < range_end_kst)
window_df = filtered_df.loc[mask_range].copy()

print({
    "rows_in_window": len(window_df),
    "start_kst": range_start_kst.strftime("%Y-%m-%d %H:%M"),
    "end_kst": range_end_kst.strftime("%Y-%m-%d %H:%M"),
})
window_df.head()


{'rows_in_window': 12, 'start_kst': '2025-01-02 04:00', 'end_kst': '2025-01-02 05:00'}


,timestamp_kst,market,timeframe,open,high,low,close,volume,timestamp_utc,ts_ms,_month_kst,date_kst
114841,2025-01-02 04:00:00,USDT/KRW,5m,1490.0,1490.9,1489.3,1489.4,39919.41311002,2025-01-01 19:00:00+00:00,1735758000000,2025-01,2025-01-02
114842,2025-01-02 04:05:00,USDT/KRW,5m,1490.0,1490.9,1489.5,1490.7,38056.87822565,2025-01-01 19:05:00+00:00,1735758300000,2025-01,2025-01-02
114843,2025-01-02 04:10:00,USDT/KRW,5m,1489.6,1490.6,1488.5,1489.9,39786.8366882,2025-01-01 19:10:00+00:00,1735758600000,2025-01,2025-01-02
114844,2025-01-02 04:15:00,USDT/KRW,5m,1489.9,1490.2,1488.8,1489.1,29614.61124547,2025-01-01 19:15:00+00:00,1735758900000,2025-01,2025-01-02
114845,2025-01-02 04:20:00,USDT/KRW,5m,1489.1,1490.3,1489.0,1489.1,14961.62999918,2025-01-01 19:20:00+00:00,1735759200000,2025-01,2025-01-02


## Median and average

In [21]:
# Compute statistics on 'close' after exclusion

# Ensure 'close' is numeric
for col in ("close",):
    if col in filtered_df.columns:
        filtered_df[col] = pd.to_numeric(filtered_df[col], errors="coerce")

median_close = filtered_df["close"].median()
average_close = filtered_df["close"].mean()

print(f"Median of close: {median_close}")
print(f"Average of close: {average_close}")


Median of close: 1399.0
Average of close: 1410.1763849858387


## excluding top 10%, bottom 10%

In [26]:
# drop NaNs (optional)

q_low = filtered_df["close"].quantile(0.10)
q_high = filtered_df["close"].quantile(0.90)

# exclude bottom 10% and top 10%
fd_trimmed = filtered_df[filtered_df["close"].between(q_low, q_high, inclusive="neither")].reset_index(drop=True)

median_close = fd_trimmed["close"].median()
average_close = fd_trimmed["close"].mean()
std_close = fd_trimmed["close"].std()

print ("Excluded top and bottom 10% of data")
print(f"Median of close: {median_close}")
print(f"Average of close: {average_close}")
print(f"Std of close: {std_close}")


Excluded top and bottom 10% of data
Median of close: 1399.0
Average of close: 1406.0574101148118
Std of close: 35.43066818908366


In [23]:
# Show the top 10% of 'close' values in filtered_df
print(filtered_df.shape)
print(fd_trimmed.shape)

# Choose existing timestamp column without conversion (time is already KST)
fd_trimmed_sorted = fd_trimmed.sort_values(by="close", ascending=False).copy()
fd_trimmed_sorted = fd_trimmed_sorted[["timestamp_kst", "close"]]

(181482, 12)
(144323, 12)


In [24]:
fd_trimmed_sorted

,timestamp_kst,close
91871,2025-01-02 04:10:00,1489.9
91912,2025-01-02 11:05:00,1489.9
91905,2025-01-02 07:15:00,1489.9
91894,2025-01-02 06:10:00,1489.9
93607,2025-01-15 00:25:00,1489.9
...,...,...
14139,2024-02-10 02:45:00,1354.0
14141,2024-02-10 03:45:00,1354.0
72275,2024-10-09 02:35:00,1354.0
72274,2024-10-09 02:30:00,1354.0
